In [2]:
using SDDP

In [3]:
graph = SDDP.LinearGraph(3)

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0

In [2]:
import Pkg
Pkg.add("SDDP")

    Updating registry at `C:\Users\atevs\.julia\registries\General.toml`
   Resolving package versions...
   Installed Conda ─────────────── v1.10.2
   Installed IrrationalConstants ─ v0.2.2
   Installed LoggingExtras ─────── v1.0.3
   Installed DiffRules ─────────── v1.15.1
   Installed ZMQ ───────────────── v1.2.6
   Installed JSON ──────────────── v0.21.4
   Installed Preferences ───────── v1.4.3
   Installed MutableArithmetics ── v1.4.5
   Installed Bzip2_jll ─────────── v1.0.8+1
   Installed BitFlags ──────────── v0.1.9
   Installed CodecBzip2 ────────── v0.8.3
   Installed Parsers ───────────── v2.8.1
   Installed SimpleBufferStream ── v1.1.0
   Installed OpenSSL_jll ───────── v3.0.14+0
   Installed BenchmarkTools ────── v1.5.0
   Installed SpecialFunctions ──── v2.4.0
   Installed TimerOutputs ──────── v0.5.24
   Installed JLLWrappers ───────── v1.5.0
   Installed StaticArraysCore ──── v1.4.3
   Installed ConcurrentUtilities ─ v2.4.2
   Installed NaNMath ───────────── v1.0.2
   

In [1]:
#initializing parameters
n = 100
N = 50
r = 70
R = 160
r_l = [30, 50, 70]
n_stages = 3

3

In [3]:
#parameterization
Ω_b = [30.0, 50.0]
P_b = [1 / 2, 1 / 2]
Ω_p = [10.0, 25.0]
P_p = [1 / 2, 1 / 2]

2-element Vector{Float64}:
 0.5
 0.5

In [6]:
Iterators.product(Ω_b, Ω_p) |> collect

2×2 Matrix{Tuple{Float64, Float64}}:
 (30.0, 10.0)  (30.0, 25.0)
 (50.0, 10.0)  (50.0, 25.0)

In [7]:
map(x -> x[1] * x[2], (Iterators.product(P_b, P_p) |> collect))

2×2 Matrix{Float64}:
 0.25  0.25
 0.25  0.25

In [4]:
function subproblem_builder(subproblem::Model, node::Int, Ω_b::Vector{Float64}, Ω_p::Vector{Float64}, P_b::Vector{Float64}, P_p::Vector{Float64})
    # State variables
    @variable(subproblem, 0 <= I_b <= 2 * n, SDDP.State, initial_value = 0)           #total basic custs at the end of stage t
    @variable(subproblem, 0 <= I_p <= 2 * N, SDDP.State, initial_value = 0)           #total premium custs at the end of stage t
    # Control variables
    @variables(subproblem, begin
        u[1:length(r_l)], (Bin, SDDP.State, initial_value = 0)                        #an upgrade offered at price level l
        q >= 0, (SDDP.State, initial_value = 0)                                       #overbooking limit
        y_b >= 0                                                                      #basic custs rejected
        y_p >= 0                                                                      #premium custs rejected
        x >= 0                                                                        #new accepted upgrades
        z_b >= 0                                                                      #new true basic custs
        z_p >= 0                                                                      #new true premium custs
    end)
    if stage == n_stages:
        @variables(subproblem, begin
            w_b >= 0                                                                  #basic custs without a room (in the end)
            w_p >= 0                                                                  #premium custs without a room (in the end)
            g[1:length(r_l)] >= 0                                                     #total upgrades granted at price level l
        end)
    # Random variables
    @variable(subproblem, d_b)
    @variable(subproblem, d_p)
    Ω = Iterators.product(Ω_b, Ω_p) |> collect
    P = map(x -> x[1] * x[2], (Iterators.product(P_b, P_p) |> collect))
    SDDP.parameterize(subproblem, Ω, P) do ω
        return JuMP.fix((d_b, d_p), ω)
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            d_b - y_b == z_b + x
            d_p == z_p + y_p
            x == (d_b - y_b) * p(u)  ##########IMPLEMENT p####################
            z_b == (d_b - y_b) * (1 - p(u))  ##########IMPLEMENT p####################

            I_b.out == I_b.in + z_b
            I_b.out <= n + q
            I_p.out == I_p.in + z_p + q
        end
    )
    # Stage-objective
    if node == n_stages
        @stageobjective(subproblem, 50 * thermal_generation)
    end
    return subproblem
end

subproblem_builder (generic function with 1 method)

In [7]:
using HiGHS

In [9]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
)

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


In [10]:
SDDP.train(model; iteration_limit = 10)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 3
  state variables : 1
  scenarios       : 2.70000e+01
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [7, 7]
  AffExpr in MOI.EqualTo{Float64}         : [2, 2]
  VariableRef in MOI.GreaterThan{Float64} : [5, 5]
  VariableRef in MOI.LessThan{Float64}    : [1, 2]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 2e+02]
  bounds range     [2e+02, 2e+02]
  rhs range        [2e+02, 2e+02]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------------

In [11]:
rule = SDDP.DecisionRule(model; node = 1)

A decision rule for node 1

In [12]:
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:volume => 150.0),
    noise = 50.0,
    controls_to_record = [:hydro_generation, :thermal_generation],
)

(stage_objective = 7500.0, outgoing_state = Dict(:volume => 200.0), controls = Dict(:thermal_generation => 150.0, :hydro_generation => -0.0))